# Building  Text-to-SQL capability to Amazon Athena using Amazon Bedrock

- **Use of amazon.titan-embed-text-v1 for creating embedding**
- **Use of Amazon OpenSearch as a vector database**
- **Use of anthropic.claude-v2:1 as base LLM Model**



## Contents

1. [Objective](#Objective)
1. [Background](#Background-(Problem-Description-and-Approach))
1. [Overall Workflow](#Overall-Workflow)
1. [Conclusion](#Conclusion)


## Objective

This notebook shows is to provide the code snippets within an executable flow from [this AWS Blog post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/).

## Background (Problem Description and Approach)

- **Problem statement**: 

Text-to-SQL solutions aim to generate SQL queries from natural language to enable non-technical users to access and analyze data. However, existing solutions face challenges related to ambiguity in natural language, needing to recreate capabilities for different databases, and collecting comprehensive metadata. The proposed solution in the text aims to address these challenges by incorporating metadata from AWS Glue Data Catalog, evaluating and correcting generated SQL queries using Amazon Athena feedback with multi-pass prompting, and leveraging Athena's support for diverse data sources.

- **Our approach**: 

[`RAG`] The [RAG approach](https://aws.amazon.com/what-is/retrieval-augmented-generation/) offers several advantages. First, it gives up-to-date, precise responses. Rather than relying only on fixed, outdated training data, RAG utilizes current external sources to formulate its answers. In this solution, we used RAG to increase the accuracy of table name from [AWS Glue Data Catalog](https://docs.aws.amazon.com/glue/latest/dg/catalog-and-crawler.html). 

[`Vector Store`] [Amazon OpenSearch](https://aws.amazon.com/opensearch-service/) offers three vector engines to choose from, each catering to different use cases.Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors. This code bases used [FAISS for similiarity search](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/).

[`Amazon Athena`] [Amazon Athena](https://aws.amazon.com/athena/) is a serverless, interactive analytics service built on open-source frameworks, supporting open-table and file formats. Athena provides a simplified, flexible way to analyze petabytes of data where it lives. In this solution, we used Amazon Athena as the SQL engine to 

[`Amazon Bedrock`] [Amazon Bedrock](https://aws.amazon.com/bedrock/) is a fully managed service that offers a choice of high-performing foundation models (FMs) from leading AI companies like AI21 Labs, Anthropic, Cohere, Meta, Stability AI, and Amazon via a single API, along with a broad set of capabilities you need to build generative AI applications with security, privacy, and responsible AI.

We used Bedrock with multi-step / multi-pass component which allows the LLM to correct the generated SQL query for accuracy. Here, the generated SQL is sent for syntax errors. We use Athena error messages to enrich our prompt for the LLM for more accurate and effective corrections in the generated SQL.

- *[RAG on AWS](https://aws.amazon.com/what-is/retrieval-augmented-generation/)*
- *[The langchain OpenSearch documentation](https://python.langchain.com/en/latest/ecosystem/opensearch.html)*
- *[Amazon OpenSearch service documentation](https://docs.aws.amazon.com/opensearch-service/index.html)*
- *[Amazon OpenSearch supports efficient vector](https://aws.amazon.com/about-aws/whats-new/2023/10/amazon-opensearch-service-vector-query-filters-faiss/)*



---

## Overall Workflow

**Prerequisite**

The following are prerequisites that needs to be accomplised before executing this notebook.
This Notebook can be executed via a Sagemaker instance or via a VS Code editor
- Create a role having access to bedrock, glue,athena, s3,lakeformation. 
- Assign the role to the Sagemaker instance or to the instance where VS Code editor is running
- Glue Database and tables. Provided spark notebook to create.
- An Amazon OpenSearch cluster for storing embeddings.Here Opensearch credenitals are in notebooks. However Opensearch cluster's access credentials (username and password) can be stored in AWS Secrets Mananger by following steps described [here](https://docs.aws.amazon.com/secretsmanager/latest/userguide/managing-secrets.html).

**The  workflow for this notebook is as follows:**
1. Create an S3 bucket with the name "knowledgebase-<ACCOUNT_ID>" 
    - create a folder "input" in that bucket
2. Download data from source 
    - https://developer.imdb.com/non-commercial-datasets/#titleakastsvgz and upload to S3 bucket from step 1 and into the "input" folder
    - https://developer.imdb.com/non-commercial-datasets/#titlebasicstsvgz and upload to S3 bucket from step 1 and into the "input" folder
3. Glue Steps
    - Create a glue database  "imdb_stg" 
    - Create a glue crawler "text-2-sql-crawler" with the datasource set to the S3 bucket created in step 1.   Run the crawler.
    - 2 tables should be created in Glue  data catalo.g  Make sure you are able to  query through athena. 
4.  From the Bedrock console, Create a new knowledgebase 
1. Install the required Python packages 
1. Create embedding and vector store. Do a similarity search with embeddings stored in the OpenSearch index for an input query.
1. Execute this notebook to generate sql..

## Step 1: Setup
Install the required packages.

In [ ]:
!pip3 install boto3
!pip3 install jq

!pip3 install langchain
!pip3 install langchain-community langchain-core
!pip3 install pandas
!pip3 install opensearch-py
!pip3 install langchain-aws
!pip3 install requests-aws4auth
!pip3 install botocore
!pip3 install pymysql
!pip3 install simplejson

## Step 2: Import all modules. There are some modules in other folder.

In [2]:
import boto3
from botocore.config import Config
from langchain_community.embeddings import BedrockEmbeddings
from langchain_aws import BedrockLLM
import traceback
import pymysql
import simplejson as simplejson

In [3]:
import logging 
import json
import os,sys
import re
import time
import pandas as pd
import io

In [6]:
from boto_client import Clientmodules
from llm_basemodel import LanguageModel
from athena_execution import AthenaQueryExecute
from openSearchVCEmbedding import EmbeddingBedrockOpenSearch

In [8]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

## Step 3: Checking access to Bedrock

In [11]:
session = boto3.session.Session()
bedrock_client = session.client('bedrock')
print(bedrock_client.list_foundation_models()['modelSummaries'][0])

{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large', 'modelId': 'amazon.titan-tg1-large', 'modelName': 'Titan Text Large', 'providerName': 'Amazon', 'inputModalities': ['TEXT'], 'outputModalities': ['TEXT'], 'responseStreamingSupported': True, 'customizationsSupported': [], 'inferenceTypesSupported': ['ON_DEMAND'], 'modelLifecycle': {'status': 'ACTIVE'}}


## Step 4: Invoking Athena and Bedrock

In [13]:
rqstath=AthenaQueryExecute()

athena client created 
s3 client created !!


### Step 4.1 Update the variables

In [15]:

index_name = 'bedrock-knowledge-base-default-index'  
## domain = 'https://pcnnwyicmvlypttno21f.us-east-1.aoss.amazonaws.com'
domain = 'https://pnwwg51jitljmsoy48gh.us-east-1.aoss.amazonaws.com'##-- update here with your OpenSearch domain
region = 'us-east-1' ##-- update here with your AWS region
vector_name = 'bedrock-knowledge-base-default-vector'
fieldname = 'id'
connection_name = 'Aurora connection'
   

In [16]:
ebropen2=EmbeddingBedrockOpenSearch(domain,  vector_name,  fieldname)
if ebropen2 is None:
    print("ebropen2 is null")
else:
    attrs = vars(ebropen2)
    print(', '.join("%s: %s" % item for item in attrs.items()))

bedrock runtime client created 


bedrockllm
bedrock_client: <botocore.client.BedrockRuntime object at 0x16c189b50>, language_model: <llm_basemodel.LanguageModel object at 0x16b8d8980>, llm: client=<botocore.client.BedrockRuntime object at 0x16c189b50> model_id='anthropic.claude-3-sonnet-20240229-v1:0' model_kwargs={'temperature': 0, 'top_k': 20, 'top_p': 1, 'stop_sequences': ['\n\nHuman:']}, embeddings: client=<botocore.client.BedrockRuntime object at 0x16c189b50> region_name=None credentials_profile_name=None model_id='amazon.titan-embed-text-v2:0' model_kwargs=None endpoint_url=None normalize=False, opensearch_domain_endpoint: https://pnwwg51jitljmsoy48gh.us-east-1.aoss.amazonaws.com, http_auth: <requests_aws4auth.aws4auth.AWS4Auth object at 0x16a2aee10>, vector_name: bedrock-knowledge-base-default-vector, fieldname: id


/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `BedrockEmbeddings` was deprecated in LangChain 0.2.11 and will be removed in 0.4.0. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import BedrockEmbeddings`.
  warn_deprecated(


## Step 5: Core logic
1. getEmbeddding : Take the input user query and vector search to find the schema from vector db created.
2. generate_sql: Taking the input prompt, generate sql . syntax_checker helps to check the sql syntax.


In [18]:
class RequestQueryBedrock:
    def __init__(self, ebropen2):
    
        ##self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        self.ebropen2 = ebropen2
  

        self.bedrock_client = ebropen2.bedrock_client
        if self.bedrock_client is None:
            self.bedrock_client = Clientmodules.createBedrockRuntimeClient()
        else : 
            print("the bedrock_client is not null")
        self.language_model = LanguageModel(self.bedrock_client)
        self.llm = self.language_model.llm
        
    def getOpenSearchEmbedding(self, index_name,user_query):
        vcindxdoc=self.ebropen2.getDocumentfromIndex(index_name=index_name)
        documnet=self.ebropen2.getSimilaritySearch(user_query,vcindxdoc)
        #return self.ebropen2.format_metadata(documnet)
        return self.ebropen2.get_data(documnet)

    def generateResponse(self,prompt):
        generated_response = self.llm.predict(prompt)
        return generated_response
        
    def generate_sql(self,prompt, max_attempt=4) ->str:
            """
            Generate and Validate SQL query.

            Args:
            - prompt (str): Prompt is user input and metadata from Rag to generating SQL.
            - max_attempt (int): Maximum number of attempts correct the syntax SQL.

            Returns:
            - string: Sql query is returned .
            """
            attempt = 0
            error_messages = []
            prompts = [prompt]

            while attempt < max_attempt:
                logger.info(f'Sql Generation attempt Count: {attempt+1}')
                try:
                    logger.info(f'we are in Try block to generate the sql and count is :{attempt+1}')
                    logger.info(f'Prompt is :{prompt}')
                    generated_sql = self.llm.predict(prompt)
                    logger.info(f'generated_sql is :{generated_sql}')
                    query_str = generated_sql.split("```")[1]
                    query_str = " ".join(query_str.split("\n")).strip()                    
                    sql_query = query_str[3:] if query_str.startswith("sql") else query_str
                    print(sql_query)
                    # return sql_query
                    syntaxcheckmsg=rqstath.syntax_checker(sql_query)
                    if syntaxcheckmsg=='Passed':
                        logger.info(f'syntax checked for query passed in attempt number :{attempt+1}')
                        return sql_query
                    else:
                        prompt = f"""{prompt}
                        This is syntax error: {syntaxcheckmsg}. 
                        To correct this, please generate an alternative SQL query which will correct the syntax error.
                        The updated query should take care of all the syntax issues encountered.
                        Follow the instructions mentioned above to remediate the error. 
                        Update the below SQL query to resolve the issue:
                        {sqlgenerated}
                        Make sure the updated SQL query aligns with the requirements provided in the initial question."""
                        prompts.append(prompt)
                        attempt += 1
                except Exception as e:
                    print(e)
                    logger.error('FAILED')
                    msg = str(e)
                    error_messages.append(msg)
                    attempt += 1
            return sql_query

Create an instance of  RequestQueryBedrock class

In [23]:
rqst=RequestQueryBedrock(ebropen2)

the bedrock_client is not null
bedrockllm


In [27]:
def userinput(user_query):
    logger.info(f'Searching metadata from vector store')
    # vector_search_match=rqst.getEmbeddding(user_query)
    vector_search_match=rqst.getOpenSearchEmbedding( index_name,user_query)
  
 
    details="It is important that the SQL query complies with Athena syntax. During join if column name are same please use alias ex llm.customer_id in select statement. It is also important to respect the type of columns: if a column is string, the value should be enclosed in quotes. If you are writing CTEs then include all the required columns. While concatenating a non string column, make sure cast the column to string. For date columns comparing to string , please cast the string input. Alwayws use the database name along with the table name. Always use ```sql to indicate start of query and ``` to indicate end of query"
    final_question = "\n\nHuman:"+details + vector_search_match + user_query+ "n\nAssistant:"
    print("FINAL QUESTION :::" + final_question)
    answer = rqst.generate_sql(final_question)
    return answer

## Step 6: User input in Natural Language

In [30]:
#user_query='how many titles exist '
#user_query = 'show me top 10 title by user rating'
#user_query = 'show me top 10 titles in US region'
#user_query = 'which year was a movie/title  made'
user_query = 'What is the most expensive distribution stream'
#user_query = 'What is the deviation between avg cost of projects in NFL cities to non NFL cities'

In [32]:
querygenerated=userinput(user_query)

Searching metadata from vector store
Sql Generation attempt Count: 1
we are in Try block to generate the sql and count is :1
Prompt is :

Human:It is important that the SQL query complies with Athena syntax. During join if column name are same please use alias ex llm.customer_id in select statement. It is also important to respect the type of columns: if a column is string, the value should be enclosed in quotes. If you are writing CTEs then include all the required columns. While concatenating a non string column, make sure cast the column to string. For date columns comparing to string , please cast the string input. Alwayws use the database name along with the table name. Always use ```sql to indicate start of query and ``` to indicate end of queryidentifier of the project"},{"Name":"country_id","Type":"integer","Comment":"This is the id of the country in which the project / store exists"},{"Name":"name","Type":"Text","Comment":"This is the name of the project which is also the name

FINAL QUESTION :::

Human:It is important that the SQL query complies with Athena syntax. During join if column name are same please use alias ex llm.customer_id in select statement. It is also important to respect the type of columns: if a column is string, the value should be enclosed in quotes. If you are writing CTEs then include all the required columns. While concatenating a non string column, make sure cast the column to string. For date columns comparing to string , please cast the string input. Alwayws use the database name along with the table name. Always use ```sql to indicate start of query and ``` to indicate end of queryidentifier of the project"},{"Name":"country_id","Type":"integer","Comment":"This is the id of the country in which the project / store exists"},{"Name":"name","Type":"Text","Comment":"This is the name of the project which is also the name of a retail store"},{"Name":"project_target_year","Type":"integer","Comment":"The field contains the year in which th

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
generated_sql is :To find the most expensive distribution stream, we can use the following SQL query:

```sql
WITH cte AS (
  SELECT 
    mp.id AS project_id,
    mpcd.distribution_stream,
    SUM(mbs.budget_amount) AS total_budget
  FROM urspace_staging.myprojects mp
  JOIN urspace_staging.myprojects_budgets mpb ON mp.id = mpb.project_id
  JOIN urspace_staging.myprojects_budget_scopes mbs ON mpb.id = mbs.budget_id
  JOIN urspace_staging.myprojects_custom_data mpcd ON mp.id = mpcd.IDX
  WHERE mpcd.distribution_stream IS NOT NULL
  GROUP BY mp.id, mpcd.distribution_stream
)
SELECT 
  distribution_stream,
  MAX(total_budget) AS max_budget
FROM cte
GROUP BY distribution_stream
ORDER BY max_budget DESC
LIMIT 1;
```

Explanation:

1. The CTE

 WITH cte AS (   SELECT      mp.id AS project_id,     mpcd.distribution_stream,     SUM(mbs.budget_amount) AS total_budget   FROM urspace_staging.myprojects mp   JOIN urspace_staging.myprojects_budgets mpb ON mp.id = mpb.project_id   JOIN urspace_staging.myprojects_budget_scopes mbs ON mpb.id = mbs.budget_id   JOIN urspace_staging.myprojects_custom_data mpcd ON mp.id = mpcd.IDX   WHERE mpcd.distribution_stream IS NOT NULL   GROUP BY mp.id, mpcd.distribution_stream ) SELECT    distribution_stream,   MAX(total_budget) AS max_budget FROM cte GROUP BY distribution_stream ORDER BY max_budget DESC LIMIT 1;
Inside syntax_checker  WITH cte AS (   SELECT      mp.id AS project_id,     mpcd.distribution_stream,     SUM(mbs.budget_amount) AS total_budget   FROM urspace_staging.myprojects mp   JOIN urspace_staging.myprojects_budgets mpb ON mp.id = mpb.project_id   JOIN urspace_staging.myprojects_budget_scopes mbs ON mpb.id = mbs.budget_id   JOIN urspace_staging.myprojects_custom_data mpcd ON mp.id 

syntax checked for query passed in attempt number :1


Status : {'State': 'SUCCEEDED', 'SubmissionDateTime': datetime.datetime(2024, 9, 6, 4, 38, 24, 989000, tzinfo=tzlocal()), 'CompletionDateTime': datetime.datetime(2024, 9, 6, 4, 38, 25, 667000, tzinfo=tzlocal())}


## Step 7: Sql Query and Query Execution output

In [34]:
def executeQuery(region,conn_name,query):

        client = boto3.client('glue', region_name=region)
        response = client.get_connection(Name=conn_name)

        connection_properties = response['Connection']['ConnectionProperties']
        URL = connection_properties['JDBC_CONNECTION_URL']
        url_list = URL.split("/")

        host = "{}".format(url_list[-2][:-5])
        port = url_list[-2][-4:]
        database = "{}".format(url_list[-1])
        user = "{}".format(connection_properties['USERNAME'])
        pwd = "{}".format(connection_properties['PASSWORD'])

        # print "user:{}".format(user)
        # print "pwd:{}".format(pwd)
        # print "host:{}".format(host)
        # print "port:{}".format(port)
        # print "database:{}".format(database)

        rs_conn = pymysql.connect(database=database, host=host, user=user, password=pwd)
        cur = rs_conn.cursor()
        cur.execute(query)
        columns = cur.description 
        result = [{columns[index][0]:column for index, column in enumerate(value)} for value in cur.fetchall()]

        #results = cur.fetchall()
        #rs_conn.commit()
        cur.close()
        return result

In [35]:
QueryOutput = executeQuery(region,connection_name,querygenerated)

In [36]:
for row in QueryOutput:
    print(row)

{'distribution_stream': 'MOB IR', 'max_budget': Decimal('1050000000.00')}


In [45]:
prompt = "Create a response that can be presented to a human without restating the question when he has asked the following query: "+user_query+"?  and Response from the database for the query being"+simplejson.dumps(QueryOutput);
generated_response = rqst.generateResponse(prompt)

In [46]:
print(generated_response)

The most expensive distribution stream is "MOB IR" with a maximum budget of $1,050,000,000.00.



## Cleanup

To avoid incurring future charges, delete the resources.


## Conclusion
In this notebook we were able to see how to use bedrock to deploy LLM Model to generate embeddings,then ingest those embeddings into OpenSearch and finally do a similarity search for user input to the documents (embeddings) stored in OpenSearch. Please read our [AWS Blog Post](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/) on this topic to learn more about the solution.
